In [1]:
library("readxl")

Warning message:
"package 'readxl' was built under R version 3.6.3"

In [2]:
library("lubridate")

Warning message:
"package 'lubridate' was built under R version 3.6.3"
Attaching package: 'lubridate'

The following objects are masked from 'package:base':

    date, intersect, setdiff, union



In [3]:
search()

[1] ".GlobalEnv"        "package:lubridate" "package:readxl"   
 [4] "jupyter:irkernel"  "jupyter:irkernel"  "package:stats"    
 [7] "package:graphics"  "package:grDevices" "package:utils"    
[10] "package:datasets"  "package:methods"   "Autoloads"        
[13] "package:base"

In [4]:
source("helpers.R")

In [5]:
path <- "../data/data.xlsx"

In [6]:
df <- Dataframing(path)

In [7]:
df <- df[-c(210),]

In [8]:
df$Date <- as.Date(df$Date)

In [9]:
firstYields <- df[df$Date == "2022-02-11",]

In [10]:
firstYields

,Date,10Y,7Y,5Y,3Y,2Y,1Y,6M,3M
,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
209,2022-02-11,1.941,1.94,1.856,1.728,1.505,1.029,0.692,0.359


#### Suponemos que: 
- $\alpha_1$ representa la tasa forward entre 0 y 3 meses, 
- $\alpha_2$ la tasa entre 3 meses y 1 año, 
- $\alpha_3$ la tasa entre 1 y 3 años,
- $\alpha_4$ la tasa entre 3 y 5 años ,
- $\alpha_5$ la tasa entre 5 y 10 años. 

In [11]:
today <- firstYields$Date

#### Además, para traer a valor presente usando una curva forward, se sigue el factor de descuento a un plazo T es: 

### $$ e^{-\int_{0}^{T} f(t) \,dt}$$

#### Definamos algunos objetos de utilidad. Cuántos cupones paga cada bono? Cómo ajustamos la tasa para cada pago?

In [12]:
alpha <- c(0.1,0.11,0.12,0.13,0.14)

In [13]:
alpha

[1] 0.10 0.11 0.12 0.13 0.14

In [14]:
today = firstYields$Date

In [15]:
threeMonthBond <- 100 + RateConverter(firstYields[["3M"]],3,today)*DiscountFactor(alpha, 3)
threeMonthBond
RateConverter(firstYields[["3M"]],3,today)
RateConverter(0.359,3,today)
RateConverter(firstYields[["1Y"]],12,today)


[1] 100.0866

[1] 0.08875278

[1] 0.08875278

[1] 1.043292

In [16]:
sixMonthBond <- 100 + RateConverter(firstYields[["6M"]], 6, today)*DiscountFactor(alpha, 6)

In [17]:
sixMonthBond

[1] 100.3301

In [18]:
oneYearBond <- 100 +  RateConverter(firstYields[["1Y"]], 6, today) *DiscountFactor(alpha, 6) + RateConverter(firstYields[["1Y"]], 12, today)*DiscountFactor(alpha, 12)

In [19]:
oneYearBond

[1] 101.4279

In [20]:
maturity<- 36

In [21]:
periods <- seq(6, maturity, 6)

In [22]:
threeMonthBond

[1] 100.0866

Recordemos que cada pago está dado por :  

$$\$100\times R \times \frac{ACT}{360}$$

In [23]:
singleBondPayment <- function(maturity, yield, alpha, today){
    return (RateConverter(yield, maturity,today)*DiscountFactor(alpha, maturity))
}

In [24]:
singleBondPayment(120,firstYields[["10Y"]], alpha, today)
firstYields[["10Y"]]

[1] 5.326155

[1] 1.941

In [25]:
yieldList <- list(firstYields)

In [26]:
nombres <- colnames(firstYields)

In [27]:
nombres <- nombres[-1]

In [28]:
nombres

[1] "10Y" "7Y"  "5Y"  "3Y"  "2Y"  "1Y"  "6M"  "3M"

In [29]:
nombres <- as.list(nombres)

In [30]:
names(nombres) <- c(120,84,60,36,24,12,6,3) 

In [31]:
BondValue <- function (today, maturity, yieldlist,alpha ){
    if (maturity == 3){
        return (100*DiscountFactor(alpha, maturity) + singleBondPayment(3,yieldlist[["3M"]], alpha, today))
        }
    setter <- as.character(maturity)
    periods <- seq(6, maturity, 6)
    bondSum <- sum(unlist(lapply(periods, function(x) singleBondPayment(x, yield = yieldlist[[nombres[[setter]]]],  alpha = alpha, today = today))))
    return(bondSum +100*DiscountFactor(alpha, maturity))
}



In [32]:
BondValue(today, 120, firstYields, alpha)

[1] 118.7522

In [33]:
maturities <- c(120,84,60,36,24,12,6,3) 

In [34]:
alpha <- c(0.00343939424789704, 0.0105322205923502, 0.0817121232717679, 0.159800322624318, 0.274481567907549)
bondValues <- list()
for (i in maturities){
    bondValues <- append(bondValues, BondValue(today, i, firstYields, alpha))
}

In [35]:
bondValues

[[1]]
[1] 100.9899

[[2]]
[1] 98.09315

[[3]]
[1] 100.8309

[[4]]
[1] 100.7474

[[5]]
[1] 98.57663

[[6]]
[1] 100.6777

[[7]]
[1] 99.99803

[[8]]
[1] 100.0027

In [36]:
error <-function(alpha){
    maturities <- c(3,6,12,24,36,60,84,120) 
    bondValues <- list()
    for (i in maturities){
        bondValues <- append(bondValues, BondValue(today, i, firstYields, alpha))
    }
    error <- sum((unlist(bondValues)-100)**2)
    return (error)
}

error(alpha)

[1] 8.350227

In [37]:

optim(alpha, error, lower = 0, upper= 1, method = "L-BFGS-B")

$par
[1] 0.003439394 0.010532221 0.081712123 0.159800323 0.274481568

$value
[1] 8.350227

$counts
function gradient 
      21       21 

$convergence
[1] 52

$message
[1] "ERROR: ABNORMAL_TERMINATION_IN_LNSRCH"

In [38]:
opt <- optim(alpha, error, lower = 0, upper= 1, method = "L-BFGS-B")

In [39]:
list(opt$par)

[[1]]
[1] 0.003439394 0.010532221 0.081712123 0.159800323 0.274481568

Valoración de bono de 7 años

In [95]:
alpha <- c(0.00343939424789704, 0.0105322205923502, 0.0817121232717679, 0.159800322624318, 0.274481567907549)
pvnotional <- 100 - DiscountFactor(alpha[4], 84)*100
pvnotional
couponsum <- function (c){
pvcoupon <- DiscountFactor(alpha[2], 12)*RateConverter(c, 12,today) + DiscountFactor(alpha[2], 24)*RateConverter(c,24,today) +
 DiscountFactor(alpha[3], 36)*RateConverter(c, 36,today) +DiscountFactor(alpha[3], 48)*RateConverter(c, 48,today) +
 DiscountFactor(alpha[4], 60)*RateConverter(c, 60,today) + DiscountFactor(alpha[4], 72)*RateConverter(c, 72,today) +  
 DiscountFactor(alpha[4], 84)*RateConverter(c, 84,today) + DiscountFactor(alpha[4], 84)*RateConverter(c, 84,today)
 result <-pvnotional- pvcoupon
    if (result <0){
        return (500)
    }
    else {
        return (pvnotional- pvcoupon)
    }
    
}

opti <- optim(c,couponsum,lower = 0, upper=10,  method = "Brent")
opti


[1] "2022-02-11"

[1] 67.32638

$par
[1] 3.835352

$value
[1] 1.174788e-06

$counts
function gradient 
      NA       NA 

$convergence
[1] 0

$message
NULL

Matriz de curvas 3 años en el pasado

In [ ]:

matAlphas <- list()
for ( r in 1:nrow(df)){ 
    firstYields <- df[r,]
    today <- firstYields$Date
    alpha <- c(0.00343939424789704, 0.0105322205923502, 0.0817121232717679, 0.159800322624318, 0.274481567907549)
    opt <- optim(alpha, error, lower = 0, upper= 1, method = "L-BFGS-B")
    matAlphas <- append(matAlphas, list(opt$par))
}

In [ ]:
MatofAlphas <- matrix(unlist(matAlphas), ncol = 5, byrow = TRUE)

In [ ]:
write.csv(MatofAlphas,"../data/sim.csv", row.names = FALSE)

In [125]:
MatofAlphas <- read.csv("../data/sim.csv")
i_alpha <- MatofAlphas[-nrow(MatofAlphas),]
f_alpha <- MatofAlphas[-1,]
delta_alpha <- f_alpha - i_alpha
cov_delta_a <- cov(delta_alpha)
cov_delta_a

,V1,V2,V3,V4,V5
V1,9.771328e-07,-7.073570e-07,2.228744e-07,2.053774e-07,4.229014e-07
V2,-7.073570e-07,1.112762e-06,6.264724e-07,8.720981e-08,-4.166142e-07
V3,2.228744e-07,6.264724e-07,1.087694e-05,1.898993e-05,1.497373e-05
V4,2.053774e-07,8.720981e-08,1.898993e-05,6.477341e-05,7.562438e-05
V5,4.229014e-07,-4.166142e-07,1.497373e-05,7.562438e-05,1.970531e-04


In [127]:
eigen_delta <- eigen(cov_delta_a)
A <- eigen_delta$vectors 
sqrt <- eigen_delta$values**(1/2)
B <- A*sqrt
B

-3.221540e-05,-3.010975e-05,-2.161982e-04,1.062240e-02,1.098073e-02
7.178107e-06,-8.480842e-05,-9.277103e-04,-4.214107e-03,4.058110e-03
-1.983426e-04,-7.937410e-04,-1.864125e-03,2.008804e-04,-2.337859e-04
-5.464285e-04,-1.078950e-03,5.067286e-04,-4.429352e-05,4.826334e-05
-4.740559e-04,2.198597e-04,-4.266474e-05,1.342217e-06,-2.926360e-06


In [129]:
Y <- MatofAlphas*A
Y

V1,V2,V3,V4,V5
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.000000e+00,0.000000e+00,-1.333184e-04,1.881514e-02,-0.2141663918
0.000000e+00,0.000000e+00,-3.773180e-03,-8.112788e-04,0.1005106588
0.000000e+00,0.000000e+00,-1.054658e-02,-1.239013e-02,-0.0214826169
0.000000e+00,0.000000e+00,5.984417e-03,-7.853055e-02,0.2068971694
0.000000e+00,0.000000e+00,-2.217448e-04,3.521188e-02,-0.2190847695
0.000000e+00,0.000000e+00,-2.396780e-03,-7.958888e-03,0.0321283608
0.000000e+00,0.000000e+00,-1.299649e-02,6.737609e-02,-0.0107767563
0.000000e+00,0.000000e+00,6.937991e-03,-7.817517e-02,0.0008183035
0.000000e+00,0.000000e+00,-1.265708e-03,9.274631e-03,0.2261002076


In [130]:
W <- MatofAlphas*B
W

V1,V2,V3,V4,V5
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.000000e+00,0.000000e+00,-7.897900e-07,9.864910e-06,-4.409611e-04
0.000000e+00,0.000000e+00,-7.768845e-06,-1.239586e-05,1.319633e-04
0.000000e+00,0.000000e+00,-1.384690e-05,-7.340026e-05,-1.126349e-05
0.000000e+00,0.000000e+00,3.137672e-06,-1.616916e-04,3.161267e-03
0.000000e+00,0.000000e+00,-3.388131e-06,4.623069e-05,-1.297878e-03
0.000000e+00,0.000000e+00,-1.419874e-05,-4.172901e-06,6.615117e-05
0.000000e+00,0.000000e+00,-2.675931e-05,1.029467e-03,-1.414911e-05
0.000000e+00,0.000000e+00,9.109087e-06,-4.631168e-04,4.290423e-07
0.000000e+00,0.000000e+00,-6.636197e-07,1.909614e-05,3.454678e-03


In [132]:
components <- princomp(delta_alpha)
summary(components)

Importance of components:
                           Comp.1      Comp.2      Comp.3      Comp.4
Standard deviation     0.01513178 0.005866852 0.002039071 0.001300243
Proportion of Variance 0.84958563 0.127713619 0.015427362 0.006273011
Cumulative Proportion  0.84958563 0.977299247 0.992726609 0.998999619
                             Comp.5
Standard deviation     0.0005192412
Proportion of Variance 0.0010003805
Cumulative Proportion  1.0000000000

In [134]:
cum_variance <- cumsum(components$sdev^2 / sum(components$sdev^2))
best_components <- components$scores[,1:2]